In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn
# from scipy.stats import poisson,skellam

import statsmodels.api as sm
import statsmodels.formula.api as smf

from league_team_names import epl_team_dict, bundesliga_team_dict, laliga_team_dict, seriea_teams_dict

country_group_dict = pd.read_csv('resources/worldcup_country_groups.csv', ).to_dict

In [61]:
players_df = pd.read_csv('resources/fifa-18-demo-player-dataset/CompleteDataset.csv')
players_df.drop(['Unnamed: 0', 'Flag', 'Club Logo', 'Value', 'Wage', 'Photo', 'Potential', "Special"], axis=1, inplace=True)
players_df[players_df['Nationality']=='Belgium'][['Name', 'Overall', 'Preferred Positions']]

/Users/michaelstrenk/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (23,35) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Name,Overall,Preferred Positions
7,E. Hazard,90,LW
11,K. De Bruyne,89,RM CM CAM
12,T. Courtois,89,GK
50,R. Lukaku,86,ST
56,T. Alderweireld,86,CB
59,R. Nainggolan,86,CAM CM
61,D. Mertens,86,ST LW CF
69,Y. Carrasco,85,RM LM
86,J. Vertonghen,85,CB
90,V. Kompany,85,CB


In [3]:
# positions_list = list(players_df['Preferred Positions'].apply(lambda x: x.split(' ', 1)[0]).drop_duplicates())
# positions_list

In [4]:
positions_dict = {'ST': 'Attack',
                  'RW': 'Attack',
                  'LW': 'Attack',
                  'GK': 'Defense',
                  'CDM': 'Midfield',
                  'CB': 'Defense',
                  'RM': 'Midfield',
                  'CM': 'Midfield',
                  'LM': 'Midfield',
                  'LB': 'Defense',
                  'CAM': 'Attack',
                  'RB': 'Defense',
                  'CF': 'Attack',
                  'RWB': 'Defense',
                  'LWB': 'Defense'}

In [5]:
players_df

,Name,Age,Nationality,Overall,Club,Acceleration,Aggression,Agility,Balance,Ball control,...,RB,RCB,RCM,RDM,RF,RM,RS,RW,RWB,ST
0,Cristiano Ronaldo,32,Portugal,94,Real Madrid CF,89,63,89,63,93,...,61.0,53.0,82.0,62.0,91.0,89.0,92.0,91.0,66.0,92.0
1,L. Messi,30,Argentina,93,FC Barcelona,92,48,90,95,95,...,57.0,45.0,84.0,59.0,92.0,90.0,88.0,91.0,62.0,88.0
2,Neymar,25,Brazil,92,Paris Saint-Germain,94,56,96,82,95,...,59.0,46.0,79.0,59.0,88.0,87.0,84.0,89.0,64.0,84.0
3,L. Suárez,30,Uruguay,92,FC Barcelona,88,78,86,60,91,...,64.0,58.0,80.0,65.0,88.0,85.0,88.0,87.0,68.0,88.0
4,M. Neuer,31,Germany,92,FC Bayern Munich,58,29,52,35,48,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,R. Lewandowski,28,Poland,91,FC Bayern Munich,79,80,78,80,89,...,58.0,57.0,78.0,62.0,87.0,82.0,88.0,84.0,61.0,88.0
6,De Gea,26,Spain,90,Manchester United,57,38,60,43,42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,E. Hazard,26,Belgium,90,Chelsea,93,54,93,91,92,...,59.0,47.0,81.0,61.0,87.0,87.0,82.0,88.0,64.0,82.0
8,T. Kroos,27,Germany,90,Real Madrid CF,60,60,71,69,89,...,76.0,72.0,87.0,82.0,81.0,81.0,77.0,80.0,78.0,77.0
9,G. Higuaín,29,Argentina,90,Juventus,78,50,75,69,85,...,51.0,46.0,71.0,52.0,84.0,79.0,87.0,82.0,55.0,87.0


In [6]:
overall_score_df = players_df[['Name', 'Club', 'Nationality', 'Preferred Positions', 'Overall']].copy()
overall_score_df['Preferred Positions'] = overall_score_df['Preferred Positions'].apply(lambda x: x.split(' ', 1)[0]).replace(positions_dict)

team = 'Brazil'
overall_score_df[overall_score_df['Nationality'] == team].head(14).groupby(['Nationality', 'Preferred Positions']).mean()


Overall
Nationality Preferred Positions           
Brazil      Attack               87.333333
            Defense              85.250000
            Midfield             84.666667

In [7]:
epl2018 = pd.read_csv('resources/leagues/epl2018.csv')
bundesliga2018 = pd.read_csv('resources/leagues/bundesliga2018.csv')
seriea2018 = pd.read_csv('resources/leagues/seriea2018.csv')
laliga2018 = pd.read_csv('resources/leagues/laliga2018.csv')
epl2018.head()

,Unnamed: 0,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,BbAv<2.5,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,PSCH,PSCD,PSCA
0,0,E0,11/08/17,Arsenal,Leicester,4,3,H,2,2,...,2.32,21,-1.00,1.91,1.85,2.10,2.02,1.49,4.73,7.25
1,1,E0,12/08/17,Brighton,Man City,0,2,A,0,0,...,2.27,20,1.50,1.95,1.91,2.01,1.96,11.75,6.15,1.29
2,2,E0,12/08/17,Chelsea,Burnley,2,3,A,0,3,...,2.23,20,-1.75,2.03,1.97,1.95,1.90,1.33,5.40,12.25
3,3,E0,12/08/17,Crystal Palace,Huddersfield,0,3,A,0,2,...,1.72,18,-0.75,2.10,2.05,1.86,1.83,1.79,3.56,5.51
4,4,E0,12/08/17,Everton,Stoke,1,0,H,1,0,...,1.76,19,-0.75,1.94,1.90,2.01,1.98,1.82,3.49,5.42


In [8]:
results_dict = {'H': 2, "D": 1, "A": 0}
cols = ['HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']
test_df = epl2018[cols].rename(columns={'FTHG': 'HomeGoals', 'FTAG': 'AwayGoals', 'FTR': 'Results'}).copy()
test_df['Results'] = test_df['Results'].replace(results_dict)
test_df

,HomeTeam,AwayTeam,HomeGoals,AwayGoals,Results
0,Arsenal,Leicester,4,3,2
1,Brighton,Man City,0,2,0
2,Chelsea,Burnley,2,3,0
3,Crystal Palace,Huddersfield,0,3,0
4,Everton,Stoke,1,0,2
5,Southampton,Swansea,0,0,1
6,Watford,Liverpool,3,3,1
7,West Brom,Bournemouth,1,0,2
8,Man United,West Ham,4,0,2
9,Newcastle,Tottenham,0,2,0


In [9]:
player_overall_df = players_df[['Name', 'Club', 'Nationality', 'Preferred Positions', 'Overall']].copy()
player_overall_df['Preferred Positions'] = player_overall_df['Preferred Positions'].apply(lambda x: x.split(' ', 1)[0]).replace(positions_dict)

In [10]:
def get_team_stats(team, type='Club', no_of_players=13, return_df=False):
    mask = overall_score_df[type] == team
    groupby_cols = [type, 'Preferred Positions']
    team_stats_df = player_overall_df[mask].head(no_of_players).groupby(groupby_cols).mean().round(0)
    if return_df == True:
        return team_stats_df
    else:
        df = team_stats_df.reset_index().pivot(type, 'Preferred Positions')
        attack = df['Overall'].values[0][0]
        mid = df['Overall'].values[0][1]
        defense = df['Overall'].values[0][2]
        return [attack, mid, defense]

In [11]:
get_team_stats('Brazil', type='Nationality', return_df=False)

[87.0, 86.0, 85.0]

In [12]:
get_team_stats('Chelsea')

[86.0, 85.0, 84.0]

In [13]:
# epl_teams = list(test_df['HomeTeam'].drop_duplicates().values)
# epl_teams

In [14]:
# epl_team_dict = {'Arsenal': 'Arsenal',
#                  'Brighton': 'Brighton & Hove Albion',
#                  'Chelsea': 'Chelsea',
#                  'Crystal Palace': 'Crystal Palace',
#                  'Everton': 'Everton',
#                  'Southampton': 'Southampton',
#                  'Watford': 'Watford',
#                  'West Brom': 'West Bromwich Albion',
#                  'Man United': 'Manchester United',
#                  'Newcastle': 'Newcastle United',
#                  'Bournemouth': 'Bournemouth',
#                  'Burnley': 'Burnley',
#                  'Leicester': 'Leicester City',
#                  'Liverpool': 'Liverpool',
#                  'Stoke': 'Stoke City',
#                  'Swansea': 'Swansea City',
#                  'Huddersfield': 'Huddersfield Town',
#                  'Tottenham': 'Tottenham Hotspur',
#                  'Man City': 'Manchester City',
#                  'West Ham': 'West Ham United'}

In [15]:
# bund_teams = list(bundesliga2018['HomeTeam'].drop_duplicates().values)
# bund_teams

In [16]:
# bundesliga_team_dict = {'Bayern Munich': 'FC Bayern Munich',
#                         'Hamburg': 'Hamburger SV',
#                         'Hertha': 'Hertha BSC Berlin',
#                         'Hoffenheim': 'TSG 1899 Hoffenheim',
#                         'Mainz': '1. FSV Mainz 05',
#                         'Schalke 04': 'FC Schalke 04',
#                         'Wolfsburg': 'VfL Wolfsburg',
#                         'Freiburg': 'SC Freiburg',
#                         "M'gladbach": 'Borussia Mönchengladbach',
#                         'FC Koln': '1. FC Köln',
#                         'Augsburg': 'FC Augsburg',
#                         'Dortmund': 'Borussia Dortmund',
#                         'Ein Frankfurt': 'Eintracht Frankfurt',
#                         'Leverkusen': 'Bayer 04 Leverkusen',
#                         'Stuttgart': 'VfB Stuttgart',
#                         'Werder Bremen': 'SV Werder Bremen',
#                         'Hannover': 'Hannover 96',
#                         'RB Leipzig': 'RB Leipzig'}

In [17]:
# laliga_teams = list(laliga2018['HomeTeam'].drop_duplicates().values)
# laliga_teams

In [18]:
# laliga_team_dict = {'Leganes': 'CD Leganés',
#                     'Valencia': 'Valencia CF',
#                     'Celta': 'RC Celta de Vigo',
#                     'Girona': 'Girona CF',
#                     'Sevilla': 'Sevilla FC',
#                     'Ath Bilbao': 'Athletic Club de Bilbao',
#                     'Barcelona': 'FC Barcelona',
#                     'La Coruna': 'RC Deportivo de La Coruña',
#                     'Levante': 'Levante UD',
#                     'Malaga': 'Málaga CF',
#                     'Betis': 'Real Betis Balompié',
#                     'Sociedad': 'Real Sociedad',
#                     'Alaves': 'Deportivo Alavés',
#                     'Las Palmas': 'UD Las Palmas',
#                     'Eibar': 'SD Eibar',
#                     'Espanol': 'RCD Espanyol',
#                     'Getafe': 'Getafe CF',
#                     'Real Madrid': 'Real Madrid CF',
#                     'Villarreal': 'Villarreal CF',
#                     'Ath Madrid': 'Atlético Madrid'
#                     }

In [19]:
# seriea_teams = list(seriea2018['HomeTeam'].drop_duplicates().values)
# seriea_teams

In [20]:
# seriea_teams_dict = {'Juventus': 'Juventus',
#                      'Verona': 'Hellas Verona',
#                      'Atalanta': 'Atalanta',
#                      'Bologna': 'Bologna',
#                      'Crotone': 'Crotone',
#                      'Inter': 'Inter',
#                      'Lazio': 'Lazio',
#                      'Sampdoria': 'Sampdoria',
#                      'Sassuolo': 'Sassuolo',
#                      'Udinese': 'Udinese',
#                      'Benevento': 'Benevento Calcio',
#                      'Genoa': 'Genoa',
#                      'Roma': 'Roma',
#                      'Chievo': 'Chievo Verona',
#                      'Fiorentina': 'Fiorentina',
#                      'Milan': 'Milan',
#                      'Napoli': 'Napoli',
#                      'Spal': 'Ferrara (SPAL)',
#                      'Torino': 'Torino',
#                      'Cagliari': 'Cagliari'
#                     }

In [21]:
team_cols = ['HomeTeam', 'AwayTeam']
epl2018[team_cols] = epl2018[team_cols].replace(epl_team_dict)
bundesliga2018[team_cols] = bundesliga2018[team_cols].replace(bundesliga_team_dict)
seriea2018[team_cols] = seriea2018[team_cols].replace(seriea_teams_dict)
laliga2018[team_cols] = laliga2018[team_cols].replace(laliga_team_dict)

In [22]:
seriea2018['HomeTeam'].drop_duplicates().sort_values()

2             Atalanta
10    Benevento Calcio
3              Bologna
24            Cagliari
13       Chievo Verona
4              Crotone
18      Ferrara (SPAL)
15          Fiorentina
11               Genoa
1        Hellas Verona
5                Inter
0             Juventus
6                Lazio
16               Milan
17              Napoli
12                Roma
7            Sampdoria
8             Sassuolo
19              Torino
9              Udinese
Name: HomeTeam, dtype: object

In [23]:
results_dict = {'H': 'HomeWin', "D": 'Draw', "A": 'AwayWin'}
cols = ['HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']

league_results_df = pd.concat([epl2018, bundesliga2018, seriea2018, laliga2018])[cols]
league_results_df.rename(columns={'FTHG': 'HomeGoals', 'FTAG': 'AwayGoals', 'FTR': 'Result'}, inplace=True)
league_results_df['Result'] = league_results_df['Result'].replace(results_dict)
league_results_df['Goal diff'] = league_results_df['HomeGoals'] - league_results_df['AwayGoals']

league_results_df.reset_index(drop=True, inplace=True)
league_results_df

,HomeTeam,AwayTeam,HomeGoals,AwayGoals,Result,Goal diff
0,Arsenal,Leicester City,4,3,HomeWin,1
1,Brighton & Hove Albion,Manchester City,0,2,AwayWin,-2
2,Chelsea,Burnley,2,3,AwayWin,-1
3,Crystal Palace,Huddersfield Town,0,3,AwayWin,-3
4,Everton,Stoke City,1,0,HomeWin,1
5,Southampton,Swansea City,0,0,Draw,0
6,Watford,Liverpool,3,3,Draw,0
7,West Bromwich Albion,Bournemouth,1,0,HomeWin,1
8,Manchester United,West Ham United,4,0,HomeWin,4
9,Newcastle United,Tottenham Hotspur,0,2,AwayWin,-2


In [24]:
# Chart this

In [25]:
league_results_df['Result'].value_counts()

HomeWin    655
AwayWin    440
Draw       351
Name: Result, dtype: int64

In [26]:
homeTeam_attack = []
homeTeam_mid = []
homeTeam_def = []
awayTeam_attack = []
awayTeam_mid = []
awayTeam_def = []
data = []


for index, row in league_results_df.iterrows():
    try:
        home_team_stats = get_team_stats(row['HomeTeam'])
        home_attack = home_team_stats[0]
        home_midfield = home_team_stats[1]
        home_defense = home_team_stats[2]

        away_team_stats = get_team_stats(row['AwayTeam'])
        away_attack = away_team_stats[0]
        away_midfield = away_team_stats[1]
        away_defense = away_team_stats[2]
                
        homeTeam_attack.append(home_attack)
        homeTeam_mid.append(home_midfield)
        homeTeam_def.append(home_defense)
        awayTeam_attack.append(away_attack)
        awayTeam_mid.append(away_midfield)
        awayTeam_def.append(away_defense)

        
    except IndexError:
        homeTeam_attack.append(None)
        homeTeam_mid.append(None)
        homeTeam_def.append(None)
        awayTeam_attack.append(None)
        awayTeam_mid.append(None)
        awayTeam_def.append(None)
        continue

In [27]:
data = [homeTeam_attack, 
        homeTeam_mid, 
        homeTeam_def, 
        awayTeam_attack,
        awayTeam_mid, 
        awayTeam_def]

feature_cols = ['homeTeam_attack', 
                'homeTeam_mid', 
                'homeTeam_def', 
                'awayTeam_attack', 
                'awayTeam_mid',
                'awayTeam_def']

home_cols = ['homeTeam_attack', 
                'homeTeam_mid', 
                'homeTeam_def', 
                'awayTeam_attack', 
                'awayTeam_mid',
                'awayTeam_def']

away_cols = ['awayTeam_attack', 
             'awayTeam_mid',
             'awayTeam_def',
             'homeTeam_attack', 
             'homeTeam_mid', 
             'homeTeam_def']

rename_cols = ['A_Team_attack', 
               'A_Team_mid',
               'A_Team_def',
               'B_Team_attack', 
               'B_Team_mid', 
               'B_Team_def']
             

team_ratings_df = pd.DataFrame(data, index=feature_cols)

In [28]:
team_ratings_df = team_ratings_df.transpose()
team_ratings_df['HomeGoals'] = league_results_df['HomeGoals']
team_ratings_df['AwayGoals'] = league_results_df['AwayGoals']
team_ratings_df['result'] = league_results_df['Result']

In [29]:
team_ratings_df.dropna(inplace=True)

In [30]:
team_ratings_df.head()

,homeTeam_attack,homeTeam_mid,homeTeam_def,awayTeam_attack,awayTeam_mid,awayTeam_def,HomeGoals,AwayGoals,result
0,85.0,83.0,83.0,80.0,78.0,78.0,4,3,HomeWin
1,76.0,75.0,75.0,89.0,83.0,85.0,0,2,AwayWin
2,86.0,85.0,84.0,75.0,77.0,75.0,2,3,AwayWin
3,78.0,76.0,79.0,75.0,73.0,74.0,0,3,AwayWin
4,81.0,81.0,82.0,78.0,79.0,77.0,1,0,HomeWin


In [31]:
home_team_df = team_ratings_df[home_cols].copy()
home_team_df.columns = rename_cols
away_team_df = team_ratings_df[away_cols].copy()
away_team_df.columns = rename_cols

X = pd.concat([home_team_df, away_team_df]).reset_index(drop=True)

In [32]:
X.tail()

,A_Team_attack,A_Team_mid,A_Team_def,B_Team_attack,B_Team_mid,B_Team_def
2887,88.0,86.0,88.0,81.0,81.0,80.0
2888,79.0,80.0,79.0,80.0,80.0,81.0
2889,78.0,78.0,78.0,86.0,85.0,83.0
2890,80.0,80.0,80.0,86.0,85.0,86.0
2891,79.0,77.0,77.0,79.0,80.0,80.0


In [33]:
print(X.shape)
X.iloc[[0,1370]]

(2892, 6)


,A_Team_attack,A_Team_mid,A_Team_def,B_Team_attack,B_Team_mid,B_Team_def
0,85.0,83.0,83.0,80.0,78.0,78.0
1370,80.0,78.0,78.0,81.0,81.0,81.0


In [34]:
y = pd.DataFrame(pd.concat([team_ratings_df['HomeGoals'], team_ratings_df['AwayGoals']]).reset_index(drop=True), columns=['goals'])

In [35]:
print(y.shape)
y.iloc[[0,1370]]

(2892, 1)


,goals
0,4
1370,4


In [36]:
y.head()

,goals
0,4
1,0
2,2
3,0
4,1


In [37]:
data = X.join(y)

In [38]:
data.head()

,A_Team_attack,A_Team_mid,A_Team_def,B_Team_attack,B_Team_mid,B_Team_def,goals
0,85.0,83.0,83.0,80.0,78.0,78.0,4
1,76.0,75.0,75.0,89.0,83.0,85.0,0
2,86.0,85.0,84.0,75.0,77.0,75.0,2
3,78.0,76.0,79.0,75.0,73.0,74.0,0
4,81.0,81.0,82.0,78.0,79.0,77.0,1


# Poisson

In [39]:


goal_model_data = X.join(y)

goal_model_data

poisson_model = smf.glm(formula="goals ~ A_Team_attack + A_Team_mid + A_Team_def + B_Team_attack + B_Team_mid + B_Team_def", data=data, 
                        family=sm.families.Poisson()).fit()
poisson_model.summary()

/Users/michaelstrenk/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                  goals   No. Observations:                 2892
Model:                            GLM   Df Residuals:                     2885
Model Family:                 Poisson   Df Model:                            6
Link Function:                    log   Scale:                             1.0
Method:                          IRLS   Log-Likelihood:                -4221.6
Date:                Tue, 12 Jun 2018   Deviance:                       3404.4
Time:                        20:55:49   Pearson chi2:                 3.01e+03
No. Iterations:                     5                                         
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -1.6821      0.507     -3.317      0.001      -2.676      -0.688
A_Team_attack     0.0434      0.011      3.934      0.000       0.022       0.065
A_Team_mid        0.0020      0.012      0.170      0.865      -0.021       0.025
A_Team_def        0.0259      0.012      2.113      0.035       0.002       0.050
B_Team_attack    -0.0129      0.011     -1.149      0.250      -0.035       0.009
B_Team_mid       -0.0359      0.012     -2.903      0.004      -0.060      -0.012
B_Team_def        0.0020      0.013      0.155      0.877      -0.023       0.027
=================================================================================
"""

In [40]:
team_data_test1 = {'A_Team_attack': 84,
                  'A_Team_mid': 84,
                  'A_Team_def': 85,
                  'B_Team_attack': 80, 
                  'B_Team_mid': 78, 
                  'B_Team_def': 80}


poisson_model.predict(pd.DataFrame(data=team_data_test1, index=[1]))


1    1.932062
dtype: float64

In [41]:


team_data_test2 = {'A_Team_attack': 80,
                  'A_Team_mid': 78,
                  'A_Team_def': 80,
                  'B_Team_attack': 84, 
                  'B_Team_mid': 84, 
                  'B_Team_def': 85}


poisson_model.predict(pd.DataFrame(data=team_data_test2, index=[1]))

1    1.090379
dtype: float64

In [42]:
a_team_stats = get_team_stats('Brazil', type='Nationality')
print(a_team_stats)

[87.0, 86.0, 85.0]


In [43]:
def simulate_goals(game, a_team='A Team', b_team='B Team', verbose=False):
    a_goals = 0
    b_goals = 0
    outcome = 0

    random_event = np.random.random()
    counter = 0


    for index, row in pd.DataFrame(data=game).iterrows():
        for x in range(len(row)):
            counter += row[x]
            
            a_goals = index
            b_goals = x
            
            if counter > random_event:
                break
        if counter > random_event:
            break
    
    if verbose == True:
        print('Random float: {}'.format(random_event))
        print('{} goals: {}'.format(a_team, str(a_goals)))
        print('{} goals: {}\n'.format(b_team, str(b_goals)))
    
    return a_goals, b_goals


def show_game_summary(game, a_team='A Team', b_team='B Team', verbose=False):
    a_team_odds = np.sum(np.tril(game, -1))
    b_team_odds = np.sum(np.triu(game, 1))
    draw_odds = np.sum(np.diag(game))
    
    if verbose == True:
        print('{} chance of Win: {}%'.format(a_team, round(a_team_odds*100, 2)))
        print('{} chance of Win: {}%'.format(b_team, round(b_team_odds*100, 2)))
        print('Odds of a draw: {}%'.format(round(draw_odds*100, 2)))
    
    a_goals, b_goals = simulate_goals(game, a_team, b_team, verbose=verbose)
    
    return a_team_odds, b_team_odds, draw_odds, a_goals, b_goals


def simulate_match(foot_model, a_team, b_team, type='Club', max_goals=9, verbose=False):
    a_team_stats = get_team_stats(a_team, type=type)
    b_team_stats = get_team_stats(b_team, type=type)
    
    data1 = {'A_Team_attack': a_team_stats[0],
              'A_Team_mid': a_team_stats[1],
              'A_Team_def': a_team_stats[2],
              'B_Team_attack': b_team_stats[0], 
              'B_Team_mid': b_team_stats[1], 
              'B_Team_def': b_team_stats[2]}
    
    data2 = {'A_Team_attack': b_team_stats[0],
              'A_Team_mid': b_team_stats[1],
              'A_Team_def': b_team_stats[2],
              'B_Team_attack': a_team_stats[0], 
              'B_Team_mid': a_team_stats[1], 
              'B_Team_def': a_team_stats[2]}
        
    a_goals_avg = foot_model.predict(pd.DataFrame(data=data1, index=[1])).values[0]
    b_goals_avg = foot_model.predict(pd.DataFrame(data=data2, index=[1])).values[0]
    
    team_pred = [[poisson.pmf(i, team_avg) for i in range(0, max_goals+1)] for team_avg in [a_goals_avg, b_goals_avg]]
    game = (np.outer(np.array(team_pred[0]), np.array(team_pred[1])))
    show_game_summary(game, a_team, b_team, verbose=verbose)
    return game


In [59]:
game = simulate_match(poisson_model, 'Belgium', 'Germany', type='Nationality')
show_game_summary(game, 'Belgium', 'Germany', verbose=True)

Belgium chance of Win: 33.21%
Germany chance of Win: 42.5%
Odds of a draw: 24.29%
Random float: 0.6620272544730578
Belgium goals: 2
Germany goals: 1



(0.3320872363735382, 0.42498530193176426, 0.24291869883834877, 2, 1)

In [45]:
simulate_goals(game)

(1, 2)

## Getting the world cup squad Stats

In [58]:
get_team_stats('Belgium', type='Nationality', return_df=True)

Overall
Nationality Preferred Positions         
Belgium     Attack                  86.0
            Defense                 86.0
            Midfield                84.0

In [47]:
get_team_stats('Sweden', type='Nationality', return_df=True)

Overall
Nationality Preferred Positions         
Sweden      Attack                  80.0
            Defense                 78.0
            Midfield                80.0

In [48]:
schedule_df = pd.read_csv('resources/worldcup_match_schedule.csv')
schedule_df.head()

,a_team,b_team
0,Russia,Saudi Arabia
1,Egypt,Uruguay
2,Portugal,Spain
3,Morocco,Iran
4,France,Australia


In [49]:
def get_bulk_team_stats(schedule):


    a_team_attack = []
    a_team_mid = []
    a_team_def = []
    b_team_attack = []
    b_team_mid = []
    b_team_def = []
    data = []


    for index, row in schedule.iterrows():
        try:
            a_team_stats = get_team_stats(row['a_team'], type='Nationality')
            a_attack = a_team_stats[0][0]
            a_midfield = a_team_stats[0][1]
            a_defense = a_team_stats[0][2]

            b_team_stats = get_team_stats(row['b_team'], type='Nationality')
            b_attack = b_team_stats[0][0]
            b_midfield = b_team_stats[0][1]
            b_defense = b_team_stats[0][2]

            if len(a_team_stats[0]) != 3 or len(b_team_stats[0]) != 3:
                a_team_attack.append(None)
                a_team_mid.append(None)
                a_team_def.append(None)
                b_team_attack.append(None)
                b_team_mid.append(None)
                b_team_def.append(None)
            else:            
                a_team_attack.append(a_attack)
                a_team_mid.append(a_midfield)
                a_team_def.append(a_defense)
                b_team_attack.append(b_attack)
                b_team_mid.append(b_midfield)
                b_team_def.append(b_defense)


        except IndexError:
            a_team_attack.append(None)
            a_team_mid.append(None)
            a_team_def.append(None)
            b_team_attack.append(None)
            b_team_mid.append(None)
            b_team_def.append(None)
            continue

    data = [a_team_attack, 
            a_team_mid, 
            a_team_def, 
            b_team_attack,
            b_team_mid, 
            b_team_def]

    rename_cols = ['A_Team_attack', 
                   'A_Team_mid',
                   'A_Team_def',
                   'B_Team_attack', 
                   'B_Team_mid', 
                   'B_Team_def']


    return pd.DataFrame(data, index=rename_cols).transpose()

In [50]:
# mask = overall_score_df['Nationality'] == 'Serbia'
# groupby_cols = ['Nationality', 'Preferred Positions']
# player_overall_df[mask].head(13).groupby(groupby_cols).mean().round(0)

In [51]:
def get_game_summary(df):

    a_team_odds = []
    b_team_odds = []
    draw_odds = []
    a_goals_count = []
    b_goals_count = []

    for index, row in schedule_df.iterrows():
        a_team = row[0]
        b_team = row[1]

        game = simulate_match(poisson_model, a_team, b_team, type='Nationality')
        a_team_probability, b_team_probability, draw_probability, a_goals, b_goals  = show_game_summary(game)

        a_team_odds.append(a_team_probability)
        b_team_odds.append(b_team_probability)
        draw_odds.append(draw_probability)
        a_goals_count.append(a_goals)
        b_goals_count.append(b_goals)
        
    
    df['a_team_chances'] = pd.Series(a_team_odds)
    df['b_team_chances'] = pd.Series(b_team_odds)
    df['draw_chances'] = pd.Series(draw_odds)
    df['a_goals'] = pd.Series(a_goals_count)
    df['b_goals'] = pd.Series(b_goals_count)
    
    return df

In [52]:
def calculate_stage_winner(df):
    a_team_points = []
    b_team_points = []

    for index, row in group_stage_results_df.iterrows():
        win = 3
        loss = 0
        draw = 1
        if row['a_goals'] == row['b_goals']:
            a_team_points.append(draw)
            b_team_points.append(draw)
        elif row['a_goals'] > row['b_goals']:
            a_team_points.append(win)
            b_team_points.append(loss)
        elif row['a_goals'] < row['b_goals']:
            a_team_points.append(loss)
            b_team_points.append(win)


        df['a_points'] = pd.Series(a_team_points)
        df['b_points'] = pd.Series(b_team_points)
        
    return df

group_stage_results_df = get_game_summary(schedule_df)
group_stage_results_df = calculate_stage_winner(group_stage_results_df)
group_stage_results_df

,a_team,b_team,a_team_chances,b_team_chances,draw_chances,a_goals,b_goals,a_points,b_points
0,Russia,Saudi Arabia,0.554290,0.202677,0.243023,1,3,0,3
1,Egypt,Uruguay,0.128553,0.675456,0.195929,3,5,0,3
2,Portugal,Spain,0.311893,0.449925,0.238170,4,1,3,0
3,Morocco,Iran,0.632917,0.147130,0.219929,0,2,0,3
4,France,Australia,0.783094,0.075025,0.141530,4,4,1,1
5,Peru,Denmark,0.210984,0.542639,0.246369,1,1,1,1
6,Argentina,Iceland,0.823846,0.058962,0.116335,2,1,3,0
7,Croatia,Nigeria,0.597850,0.175881,0.226247,0,3,0,3
8,Brazil,Switzerland,0.611717,0.175833,0.212408,1,1,1,1
9,Costa Rica,Serbia,0.214069,0.543399,0.242522,1,2,0,3


In [53]:
cols = {'a_team': 'Team', 'a_goals': 'Goals', 'a_points': 'Points',
        'b_team': 'Team', 'b_goals': 'Goals', 'b_points': 'Points'}

a_team_group_results_df = group_stage_results_df[['a_team', 'a_goals', 'a_points']].rename(columns=cols)
b_team_group_results_df = group_stage_results_df[['b_team', 'b_goals', 'b_points']].rename(columns=cols)

scoreboard_df = pd.concat([a_team_group_results_df, b_team_group_results_df]).reset_index(drop=True)
scoreboard_df = scoreboard_df.merge(pd.read_csv('resources/worldcup_country_groups.csv'), on='Team', how='left')
scoreboard_df = scoreboard_df.groupby(['Group', 'Team']).sum().reset_index()
scoreboard_df = scoreboard_df.sort_values(['Group', 'Points', 'Goals'], ascending=[True, False, False]).reset_index(drop=True)
scoreboard_df

,Group,Team,Goals,Points
0,A,Uruguay,9,6
1,A,Russia,7,6
2,A,Egypt,6,3
3,A,Saudi Arabia,5,3
4,B,Portugal,7,7
5,B,Spain,4,6
6,B,Iran,2,3
7,B,Morocco,1,1
8,C,France,8,7
9,C,Australia,7,4


In [54]:
round_of_16_games = [(0, 5), (8, 13),
                     (16, 21), (24, 29),
                     (4, 1), (12, 9),
                     (20, 17), (28, 25)]

round_of_8_games = [(0, 1), (2, 3),
                     (4, 5), (6, 7)]

round_of_4_games = [(0, 1), (2, 3)]

round_of_2_games = [(0, 1)]

def get_team_penalties(team):
    mask = players_df['Nationality'] == team
    return pd.to_numeric(players_df[mask]['Penalties'].head(), errors='coerce').mean()

def simulate_penalties(a_team, b_team):
    a_pen = get_team_penalties(a_team)
    b_pen = get_team_penalties(b_team)
    
    if a_pen == b_pen:
        winner = a_team
    elif a_pen > b_pen:
        winner = a_team
    elif a_pen < b_pen:
        winner = b_team
    
    return winner

def get_knockout_winner(round_results_df):
    winners = []
    
    for index, row in round_results_df.iterrows():
        if row['a_goals'] == row['b_goals']:
            winner = simulate_penalties(row[0], row[1])
        elif row['a_goals'] > row['b_goals']:
            winner = row[0]
        elif row['a_goals'] < row['b_goals']:
            winner = row[1]
            
        winners.append(winner)
        
    round_results_df['Winner'] = pd.Series(winners)
    
    return round_results_df
    

def simulate_knockout_stage(df, schedule_tuples):
    a_team_lists = []
    b_team_lists = []

    a_team_odds = []
    b_team_odds = []
    draw_odds = []
    a_goals_count = []
    b_goals_count = []
    
    
    for match in schedule_tuples:
        a_team = df.iloc[match[0]]['Team']
        b_team = df.iloc[match[1]]['Team']
        a_team_lists.append(a_team)
        b_team_lists.append(b_team)

        game = simulate_match(poisson_model, a_team, b_team, type="Nationality")
        a_team_probability, b_team_probability, draw_probability, a_goals, b_goals  = show_game_summary(game)

        a_team_odds.append(a_team_probability)
        b_team_odds.append(b_team_probability)
        draw_odds.append(draw_probability)
        a_goals_count.append(a_goals)
        b_goals_count.append(b_goals)

    round_results_df = pd.DataFrame([a_team_lists, b_team_lists]).transpose()
    round_results_df = get_game_summary(round_results_df)
    round_results_df = get_knockout_winner(round_results_df)
    
    df = pd.DataFrame(round_results_df['Winner']).rename(columns={'Winner': 'Team'})    
    
    return df



In [55]:
def simulate_World_Cup(n_simulations=None, verbose=False):
    world_cup_winners = []
    n = 0
    while n < n_simulations:
        try:
            group_stage_results_df = get_game_summary(schedule_df)
            group_stage_results_df = calculate_stage_winner(group_stage_results_df)

            cols = {'a_team': 'Team', 'a_goals': 'Goals', 'a_points': 'Points',
                    'b_team': 'Team', 'b_goals': 'Goals', 'b_points': 'Points'}

            a_team_group_results_df = group_stage_results_df[['a_team', 'a_goals', 'a_points']].rename(columns=cols)
            b_team_group_results_df = group_stage_results_df[['b_team', 'b_goals', 'b_points']].rename(columns=cols)

            scoreboard_df = pd.concat([a_team_group_results_df, b_team_group_results_df]).reset_index(drop=True)
            scoreboard_df = scoreboard_df.merge(pd.read_csv('resources/worldcup_country_groups.csv'), on='Team', how='left')
            scoreboard_df = scoreboard_df.groupby(['Group', 'Team']).sum().reset_index()
            scoreboard_df = scoreboard_df.sort_values(['Group', 'Points', 'Goals'], ascending=[True, False, False])

            df_8 = simulate_knockout_stage(scoreboard_df, round_of_16_games)
            df_4 = simulate_knockout_stage(df_8, round_of_8_games)
            df_2 = simulate_knockout_stage(df_4, round_of_4_games)
            world_cup_winner = simulate_knockout_stage(df_2, round_of_2_games).iloc[0].item()

            world_cup_winners.append(world_cup_winner)
            
            if verbose == True:
                print(group_stage_results_df)
                print(scoreboard_df)
                print(df_8)
                print(df_4)
                print(df_2)
                print(pd.Series(world_cup_winners).value_counts())
                
            print('The winner of World Cup 2018 simulation {}/{} is {}'.format(n+1, n_simulations, world_cup_winner))
            

            n += 1
        except:
            print('Exception occured')
            continue
        
    return pd.Series(world_cup_winners).value_counts()
        


In [56]:
# world_cup_results = simulate_World_Cup(1000, verbose=False)

The winner of World Cup 2018 simulation 1/1000 is Belgium
The winner of World Cup 2018 simulation 2/1000 is Brazil
The winner of World Cup 2018 simulation 3/1000 is Spain
The winner of World Cup 2018 simulation 4/1000 is Uruguay
The winner of World Cup 2018 simulation 5/1000 is Morocco
The winner of World Cup 2018 simulation 6/1000 is England
The winner of World Cup 2018 simulation 7/1000 is Belgium
The winner of World Cup 2018 simulation 8/1000 is England
The winner of World Cup 2018 simulation 9/1000 is Iceland
The winner of World Cup 2018 simulation 10/1000 is Colombia
The winner of World Cup 2018 simulation 11/1000 is Uruguay
The winner of World Cup 2018 simulation 12/1000 is Argentina
The winner of World Cup 2018 simulation 13/1000 is Argentina
The winner of World Cup 2018 simulation 14/1000 is Argentina
The winner of World Cup 2018 simulation 15/1000 is Portugal
The winner of World Cup 2018 simulation 16/1000 is England
The winner of World Cup 2018 simulation 17/1000 is Portugal


The winner of World Cup 2018 simulation 140/1000 is Sweden
The winner of World Cup 2018 simulation 141/1000 is Argentina
The winner of World Cup 2018 simulation 142/1000 is England
The winner of World Cup 2018 simulation 143/1000 is Croatia
The winner of World Cup 2018 simulation 144/1000 is Japan
The winner of World Cup 2018 simulation 145/1000 is Mexico
The winner of World Cup 2018 simulation 146/1000 is Argentina
The winner of World Cup 2018 simulation 147/1000 is Egypt
The winner of World Cup 2018 simulation 148/1000 is Japan
The winner of World Cup 2018 simulation 149/1000 is Argentina
The winner of World Cup 2018 simulation 150/1000 is Nigeria
The winner of World Cup 2018 simulation 151/1000 is Colombia
The winner of World Cup 2018 simulation 152/1000 is Croatia
The winner of World Cup 2018 simulation 153/1000 is Belgium
The winner of World Cup 2018 simulation 154/1000 is Portugal
The winner of World Cup 2018 simulation 155/1000 is Mexico
The winner of World Cup 2018 simulation 1

The winner of World Cup 2018 simulation 277/1000 is Sweden
The winner of World Cup 2018 simulation 278/1000 is Argentina
The winner of World Cup 2018 simulation 279/1000 is Sweden
The winner of World Cup 2018 simulation 280/1000 is Uruguay
The winner of World Cup 2018 simulation 281/1000 is Mexico
The winner of World Cup 2018 simulation 282/1000 is Croatia
The winner of World Cup 2018 simulation 283/1000 is Sweden
The winner of World Cup 2018 simulation 284/1000 is Germany
The winner of World Cup 2018 simulation 285/1000 is Germany
The winner of World Cup 2018 simulation 286/1000 is Argentina
The winner of World Cup 2018 simulation 287/1000 is Argentina
The winner of World Cup 2018 simulation 288/1000 is Spain
The winner of World Cup 2018 simulation 289/1000 is Mexico
The winner of World Cup 2018 simulation 290/1000 is Argentina
The winner of World Cup 2018 simulation 291/1000 is Egypt
The winner of World Cup 2018 simulation 292/1000 is Portugal
The winner of World Cup 2018 simulation 

The winner of World Cup 2018 simulation 414/1000 is Mexico
The winner of World Cup 2018 simulation 415/1000 is Belgium
The winner of World Cup 2018 simulation 416/1000 is Argentina
The winner of World Cup 2018 simulation 417/1000 is Croatia
The winner of World Cup 2018 simulation 418/1000 is Brazil
The winner of World Cup 2018 simulation 419/1000 is Belgium
The winner of World Cup 2018 simulation 420/1000 is Saudi Arabia
The winner of World Cup 2018 simulation 421/1000 is Germany
The winner of World Cup 2018 simulation 422/1000 is Germany
The winner of World Cup 2018 simulation 423/1000 is Iceland
The winner of World Cup 2018 simulation 424/1000 is Poland
The winner of World Cup 2018 simulation 425/1000 is Mexico
The winner of World Cup 2018 simulation 426/1000 is Croatia
The winner of World Cup 2018 simulation 427/1000 is Argentina
The winner of World Cup 2018 simulation 428/1000 is England
The winner of World Cup 2018 simulation 429/1000 is Belgium
The winner of World Cup 2018 simula

The winner of World Cup 2018 simulation 551/1000 is Croatia
The winner of World Cup 2018 simulation 552/1000 is Colombia
The winner of World Cup 2018 simulation 553/1000 is Mexico
The winner of World Cup 2018 simulation 554/1000 is Argentina
The winner of World Cup 2018 simulation 555/1000 is Croatia
The winner of World Cup 2018 simulation 556/1000 is Croatia
The winner of World Cup 2018 simulation 557/1000 is Mexico
The winner of World Cup 2018 simulation 558/1000 is Portugal
The winner of World Cup 2018 simulation 559/1000 is Brazil
The winner of World Cup 2018 simulation 560/1000 is Sweden
The winner of World Cup 2018 simulation 561/1000 is France
The winner of World Cup 2018 simulation 562/1000 is Colombia
The winner of World Cup 2018 simulation 563/1000 is Morocco
The winner of World Cup 2018 simulation 564/1000 is Sweden
The winner of World Cup 2018 simulation 565/1000 is Spain
The winner of World Cup 2018 simulation 566/1000 is Korea Republic
The winner of World Cup 2018 simulat

The winner of World Cup 2018 simulation 688/1000 is Germany
The winner of World Cup 2018 simulation 689/1000 is Argentina
The winner of World Cup 2018 simulation 690/1000 is Uruguay
The winner of World Cup 2018 simulation 691/1000 is England
The winner of World Cup 2018 simulation 692/1000 is Argentina
The winner of World Cup 2018 simulation 693/1000 is Argentina
The winner of World Cup 2018 simulation 694/1000 is Spain
The winner of World Cup 2018 simulation 695/1000 is Argentina
The winner of World Cup 2018 simulation 696/1000 is Morocco
The winner of World Cup 2018 simulation 697/1000 is Mexico
The winner of World Cup 2018 simulation 698/1000 is Sweden
The winner of World Cup 2018 simulation 699/1000 is Portugal
The winner of World Cup 2018 simulation 700/1000 is Iceland
The winner of World Cup 2018 simulation 701/1000 is Germany
The winner of World Cup 2018 simulation 702/1000 is Belgium
The winner of World Cup 2018 simulation 703/1000 is Uruguay
The winner of World Cup 2018 simula

The winner of World Cup 2018 simulation 825/1000 is Morocco
The winner of World Cup 2018 simulation 826/1000 is England
The winner of World Cup 2018 simulation 827/1000 is Sweden
The winner of World Cup 2018 simulation 828/1000 is Egypt
The winner of World Cup 2018 simulation 829/1000 is Germany
The winner of World Cup 2018 simulation 830/1000 is Denmark
The winner of World Cup 2018 simulation 831/1000 is Uruguay
The winner of World Cup 2018 simulation 832/1000 is Portugal
The winner of World Cup 2018 simulation 833/1000 is Mexico
The winner of World Cup 2018 simulation 834/1000 is Mexico
The winner of World Cup 2018 simulation 835/1000 is Argentina
The winner of World Cup 2018 simulation 836/1000 is Morocco
The winner of World Cup 2018 simulation 837/1000 is Uruguay
The winner of World Cup 2018 simulation 838/1000 is Colombia
The winner of World Cup 2018 simulation 839/1000 is Germany
The winner of World Cup 2018 simulation 840/1000 is Egypt
The winner of World Cup 2018 simulation 841

The winner of World Cup 2018 simulation 962/1000 is Argentina
The winner of World Cup 2018 simulation 963/1000 is Colombia
The winner of World Cup 2018 simulation 964/1000 is Japan
The winner of World Cup 2018 simulation 965/1000 is Argentina
The winner of World Cup 2018 simulation 966/1000 is Germany
The winner of World Cup 2018 simulation 967/1000 is Nigeria
The winner of World Cup 2018 simulation 968/1000 is France
The winner of World Cup 2018 simulation 969/1000 is Belgium
The winner of World Cup 2018 simulation 970/1000 is Argentina
The winner of World Cup 2018 simulation 971/1000 is Belgium
The winner of World Cup 2018 simulation 972/1000 is Mexico
The winner of World Cup 2018 simulation 973/1000 is Mexico
The winner of World Cup 2018 simulation 974/1000 is Spain
The winner of World Cup 2018 simulation 975/1000 is Argentina
The winner of World Cup 2018 simulation 976/1000 is Sweden
The winner of World Cup 2018 simulation 977/1000 is France
The winner of World Cup 2018 simulation 

In [57]:
world_cup_results

Argentina         127
Belgium            88
Germany            87
England            86
Croatia            73
Mexico             60
Sweden             57
Uruguay            56
Portugal           54
Colombia           54
Spain              43
Morocco            33
Egypt              23
Japan              23
Russia             23
Brazil             23
Iceland            16
France             14
Nigeria            12
Poland              9
Senegal             8
Korea Republic      7
Saudi Arabia        6
Denmark             5
Serbia              4
Peru                4
Iran                3
Costa Rica          2
dtype: int64